<a href="https://colab.research.google.com/github/ahatasham5/Bangla_Handwritting_CSE366/blob/main/CNN_Bangla_Handwritting_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the notebook you have seen in the overview video.

## Creating the CNN model

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
import zipfile
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive')

# Replace '/content/drive/MyDrive/archive2.zip' with the actual path to your zip file on Google Drive
with zipfile.ZipFile('/content/drive/MyDrive/HWrec/archive3.zip', 'r') as zip_ref:
    # Extract all the contents of the zip file in the current directory
    zip_ref.extractall()

Mounted at /content/drive


In [3]:
!pwd


/content


In [4]:
classifier = Sequential()

classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', input_shape = (40, 40, 3)))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(.2))

classifier.add(Dense(units = 10, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Fitting the CNN model to the Dataset

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile

In [6]:
!pwd
!ls
!ls /content/archive3/Train

/content
archive3  drive  sample_data
0  1  2  3  4  5  6  7	8  9


Data Generate

In [7]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = .2, rotation_range = 25)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/archive3/Test', target_size = (40, 40),
                                                 batch_size = 32, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/archive3/Train', target_size = (40, 40),
                                                 batch_size = 32, class_mode = 'categorical')

Found 14724 images belonging to 10 classes.
Found 1120 images belonging to 10 classes.


In [8]:
import numpy as np

# Get a single batch of images and labels
images_batch, labels_batch = next(training_set)

# Check if there are any NaN or Inf values in the images
if np.isnan(images_batch).any():
    print('Images contain NaN values.')
if np.isinf(images_batch).any():
    print('Images contain Inf values.')

In [9]:
# Get a single batch of images and labels
images_batch, labels_batch = next(training_set)

# Print the shape of the images
print('Shape of images:', images_batch.shape)


Shape of images: (32, 40, 40, 3)


In [10]:
from keras.callbacks import EarlyStopping

# Define the number of times to repeat the data generator
num_epochs = 3
num_repeats = 1  # Adjust as needed

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

for _ in range(num_epochs):
    for _ in range(num_repeats):
        # Reset the data generator before each repeat
        training_set.reset()

        # Train for one epoch
        classifier.fit(training_set, steps_per_epoch=len(training_set), epochs=1, validation_data=test_set,
                       validation_steps=len(test_set), callbacks=[early_stopping])


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


461/461 ━━━━━━━━━━━━━━━━━━━━ 32s 55ms/step - accuracy: 0.6537 - loss: 1.0223 - val_accuracy: 0.9161 - val_loss: 0.2925
461/461 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.8925 - loss: 0.3221 - val_accuracy: 0.9393 - val_loss: 0.2267
461/461 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.9270 - loss: 0.2228 - val_accuracy: 0.9509 - val_loss: 0.1744


In [11]:
classifier.fit(training_set, steps_per_epoch = 35, epochs = 20,
               validation_data = test_set, validation_steps = 460)


Epoch 1/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9373 - loss: 0.2126

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - accuracy: 0.9373 - loss: 0.2128 - val_accuracy: 0.9482 - val_loss: 0.1831
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.9378 - loss: 0.1809 - val_accuracy: 0.9438 - val_loss: 0.1879
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.9220 - loss: 0.2329 - val_accuracy: 0.9402 - val_loss: 0.1977
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.9257 - loss: 0.2009 - val_accuracy: 0.9446 - val_loss: 0.2022
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9503 - loss: 0.1862 - val_accuracy: 0.9563 - val_loss: 0.1768
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.9137 - loss: 0.2411 - val_accuracy: 0.9536 - val_loss: 0.1736
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.9229 - loss: 0.2056 - val_accuracy: 0.9411 - val_loss: 0.2001
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.9335 - loss: 0.1897 - val_accuracy: 0.9464 - val_loss: 

In [12]:
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 38, 38, 128)         │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 19, 19, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 19, 19, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 17, 17, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         524,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,809,248 (6.90 MB)

 Trainable params: 603,082 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,206,166 (4.60 MB)

## Creating a graphical user interface to draw the character

In [42]:
from IPython.display import display, HTML, Javascript
import io
import base64
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets




# Create an HTML canvas
canvas_html = """
<canvas width="400" height="400" id="drawing-canvas" style="border:1px solid black;"></canvas>
"""

# Create JavaScript code to handle mouse events and update the canvas
javascript = """
<script type="text/javascript">
    // Get the canvas element and its context
    var canvas = document.getElementById('drawing-canvas');
    var ctx = canvas.getContext('2d');

    // Set the initial background to white ONCE
    function initializeCanvas() {
        ctx.fillStyle = 'white';
        ctx.fillRect(0, 0, canvas.width, canvas.height);
    }

    initializeCanvas(); // Call this to set the white background at the start

    // Set the stroke style for drawing
    ctx.strokeStyle = 'black';
    ctx.lineWidth = 10;

    // Flag to indicate whether the mouse button is pressed
    var isDrawing = false;

    // Function to handle mouse down events
    function handleMouseDown(event) {
        // Start drawing
        isDrawing = true;
        ctx.beginPath();
        ctx.moveTo(event.offsetX, event.offsetY);
    }

    // Function to handle mouse move events
    function handleMouseMove(event) {
        if (isDrawing) {
            // Draw a line to the current mouse position
            ctx.lineTo(event.offsetX, event.offsetY);
            ctx.stroke();
        }
    }

    // Function to handle mouse up events
    function handleMouseUp(event) {
        // Stop drawing
        isDrawing = false;
    }

    // Register event listeners
    canvas.addEventListener('mousedown', handleMouseDown);
    canvas.addEventListener('mousemove', handleMouseMove);
    canvas.addEventListener('mouseup', handleMouseUp);
</script>

"""

# Display the canvas and run the JavaScript code
display(HTML(canvas_html + javascript))

def determine_character(res):
    if res == 0:
        print('prediction : 0')
    elif res == 1:
        print('prediction : 1')
    elif res == 2:
        print('prediction : 2')
    elif res == 3:
        print('prediction : 3')
    elif res == 4:
        print('prediction : 4')
    elif res == 5:
        print('prediction : 5')
    elif res == 6:
        print('prediction : 6')
    elif res == 7:
        print('prediction : 7')
    elif res == 8:
        print('prediction : 8')
    elif res == 9:
        print('prediction : 9')

def single_prediction(test_img):
    test_img_arr = image.img_to_array(test_img)
    test_img_arr = np.expand_dims(test_img_arr, axis = 0)
    prediction = classifier.predict(test_img_arr)
    result = np.argmax(prediction, axis = 1)
    determine_character(result)

from google.colab import output

def predict_from_canvas():
    # Execute the JavaScript function to get image data
    get_image_data_js = """
    async function getImageData() {
        const canvas = document.getElementById('drawing-canvas');
        const ctx = canvas.getContext('2d');

        // Get the current canvas content as an image
        const imgData = ctx.getImageData(0, 0, canvas.width, canvas.height);

        // Convert image data to JSON-friendly format
        return {
            data: Array.from(imgData.data), // Convert Uint8ClampedArray to plain array
            width: canvas.width,
            height: canvas.height
        };
    }
    getImageData();

    """

    # Get data from JavaScript
    data = output.eval_js(get_image_data_js)

    # Validate data dimensions
    print(f"Canvas Width: {data['width']}, Height: {data['height']}")
    print(f"Image Data Length: {len(data['data'])}")

    # Convert the data into a NumPy array
    img_array = np.array(data['data'], dtype=np.uint8).reshape((data['height'], data['width'], 4))

    # Debug: Display the first few pixels
    print(f"Sample Pixels: {img_array[0, :5, :]}")

    print(f"Image Array Shape: {img_array.shape}")
    print(f"Sample Pixels (First Row): {img_array[0, :5]}")

    # Convert RGBA to RGB
    img = Image.fromarray(img_array, 'RGBA').convert('RGB')

    # Resize the image to the required input size for the model
    img = img.resize((40, 40))

    # Save the image
    img_path = '/content/drive/MyDrive/drawn_images/drawn_image.png'
    img.save(img_path)

    #debag image
    debug_path = '/content/drive/MyDrive/drawn_images/debug_image.png'
    debug_img = Image.fromarray(img_array, 'RGBA')
    debug_img.save(debug_path)
    print(f"Debug image saved to {debug_path}")

    # Pass the image to your model
    single_prediction(img)


    # Create a button to make a prediction
predict_button = widgets.Button(description='Predict')

# Function to handle button click
def on_predict_button_click(b):
    predict_from_canvas()

# Register event handler
predict_button.on_click(on_predict_button_click)

# Display the button
display(predict_button)

Button(description='Predict', style=ButtonStyle())

Canvas Width: 400, Height: 400
Image Data Length: 640000
Sample Pixels: [[255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]]
Image Array Shape: (400, 400, 4)
Sample Pixels (First Row): [[255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]]
Debug image saved to /content/drive/MyDrive/drawn_images/debug_image.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
prediction : 9
